<h2>Assignment 2 - Content Based Recommender</h2>
<p>David Flanagan INFO-T780<br>May 9th, 2019</p>

<h3>Running the Assignment</h3>
<p>This assignment was implemented in a jupyter notebook as requested.  The code requires the following libraries:</p>
<ul>
<li>numpy</li>
<li>jupyter</li>
<li>tabulate</li>
</ul>
<p>All of the listed packages area available through pip. To run the assignment clone the repository via</p>
<code>git clone https://github.com/df424/drexel_INFO-T780</code>
<p>The navigate to the repository directory and run</p>
<code>jupyter notebook</code>
<p>Assuming everything has been installed correctly the jupyter server should start and you can then click on "assignment2.ipynb" in your webbrowser to run the assignment interactively.</p>

<h3>About the Data</h3>
<p>In this assignment I created a content based recommender system using a dataset I downloaded from IMDB.  The data site is available for download <a href="https://datasets.imdbws.com/">here</a>.</p>
<p>The data set contains 1000 movies and while the data set has many features I narrowed them down to the following:
    <ol>
        <li>Genre</li>
        <li>IMDB Rating</li>
        <li>Year Released</li>
        <li>Runtime</li>
        <li>Number of people who rated the movie</li>
        <li>Revenue in millions of dollars</li>
        <li>Metascore from across multiple ratings sites</li>
    </ol>
</p>

In [11]:
# Imports
import clean_data as cd
import movie_recommender as mr
import numpy as np
import ipywidgets as widgets
from IPython.display import HTML, display
import tabulate
EXTRACTED_DATA_PATH = 'IMDB-Movie-Data.csv'
ZIPPED_DATA_PATH = 'data/imdb_data.zip'

<h3>Data Processing</h3>
<p>This cell unzips the data file and process it extensively.<br>
The genres are turned into a vector of boolean values where the value is set to 1 (True) if the movie is listed under that genre or false otherwise.  All of the other parameters are processed as ordinal data.  They are converted to discrete values based on range.</p>
<p>
The available genres are as follows: Action, Western, Music, History, Thriller, Drama, Horror, Musical, Animation, War, Fantasy, Comedy, Adventure, Mystery Sci-Fi, Sport, Family, Biography, Crime, and Romance</p>
<p>The features are stored in the data matrix X with each row representing a movie.  The shape of the data is 1000x26. The cooresponding labels are stored in the column vector Y.</p>

 

In [12]:
#Clean the data
cd.unzipData(ZIPPED_DATA_PATH)
X, Y = cd.loadData(EXTRACTED_DATA_PATH)

<h3>Creation of Recommender System</h3>
<p>This cell actually creates the recommender system and sets up the weight vectors.  The weight vector was set via "knowledge elicitation" from a self declared expert
in what people want to watch.</p>
<p>All of the genres are given a weight of 0.7 since the expert considered genre an important feature when selecting a movie.  Likewise the IMDB rating as well as the metascore were given weights of 0.9 and 0.8 respectively since these features are considered important.  All other features were given a value between 0.1 and 0.3 as they are considered less important but should still have an impact.</p>
<p>This section also sets up the local similarity functions.  Two different location functions are used one to compare the genres and the other to compare the ordinal data.  The genre comparison function returns a 1 if and only if the input vector has that genre selected and the movie vector is of that genre.  In otherwords it is essentally a logical AND gate.  Think of it as adding to the similarity if the input vector and the movie both have the genre listed, but no increase in similarity is added just because both the movie and the input vector both do not have a genre listed.</p>
<p>The local similarity function for the ordinal data is given as follows:</p>

$$ordinal_similarity = 1-\frac{abs(X^{(i)}-x_i)}{range(X^{(i)})}$$

<p>Where $X^{(i)}$ is the collumn of feature $i$ and $x_i$ is the $i$th feature of the input vector.</p>

In [13]:
#Setup the difference functions, weights and recommender system.
weights = np.array([0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 
                    0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 
                    0.7, 0.7, 0.1, 0.2, 0.9, 0.1, 0.2, 0.8])

difference_fxs = []
for i in range(X.shape[1]):
    if i >= 20:
        difference_fxs.append(mr.absDifference)
    else:
        difference_fxs.append(mr.genreCompare)

recommender = mr.MovieRecommenderSystem(X,Y, weights, difference_fxs)

<h3>Input Vector</h3>
<p>Use these widgets to select the input vector to run kNN against.  We use radio buttons for most of the features.  However, the genre is a multiple select box so that we can select as many genres as we want to include in the recommendation query.</p>

In [14]:
ratingWidget = widgets.RadioButtons(description='IMDB Rating:', options=cd.ratingDict.keys())
genreWidget = widgets.SelectMultiple(description='Genres:', options=cd.genreDict.keys())
lengthWidget = widgets.RadioButtons(description='Movie Length:', options=cd.lengthDict.keys())
releaseDateWidget = widgets.RadioButtons(description="Release Date:", options=cd.releaseDateDict.keys())
ratingCountWidget = widgets.RadioButtons(description='Number of Ratings:', options=cd.ratingCountDict.keys())
revenueWidget = widgets.RadioButtons(description='Revenue:', options=cd.revenueDict.keys())
metascoreWidget = widgets.RadioButtons(description='MetaScore:', options=cd.metaScoreDict.keys())
display(widgets.HBox([genreWidget, releaseDateWidget, lengthWidget]),
        widgets.HBox([ratingWidget, ratingCountWidget, revenueWidget]), metascoreWidget)


RadioButtons(description='MetaScore:', options=('Very High (>90)', 'High (70-90)', 'Average (50-70)', 'Low (30…

<h3>Example Output</h3>
<p>In this cell the widgets are turned into the input vector.  I print the input vector so you can see a concrete example of what a row of the data would look like.</p>
<p>The data is then displayed in a table.  Notice how above I have selected Fantasy, Adventure and War. It doesn't seem any examples have all three, but Fantasy and Adventure is highly represented.  Likewise We can see that all the movies are from the date range, revenues and ratings that I selected in the widgets above.  It seems to be working quite well.  I ran several other queries that also seemed to match the input, but didn't display them here because of the logistics of putting that repeatedly in the jupyter notebook.</p>

In [23]:
input_vec = cd.selectionToInputVector(genreWidget.value, 
                                      releaseDateWidget.value, 
                                      lengthWidget.value, 
                                      ratingWidget.value, 
                                      ratingCountWidget.value, 
                                      revenueWidget.value, 
                                      metascoreWidget.value)

print(input_vec)

# Get the top 5 recommendations.
recommendations = recommender.getRecommendations(input_vec, 5)

# display the data in jupyter as an html table.
display(HTML(tabulate.tabulate(recommendations, tablefmt='html')))

[0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 3. 3. 1.
 3. 3.]


51,Star Wars: Episode VII - The Force Awakens,"Action,Adventure,Fantasy","Three decades after the defeat of the Galactic Empire, a new threat arises. The First Order attempts to rule the galaxy and only a ragtag group of heroes can stop them, along with the help of the Resistance.",J.J. Abrams,"Daisy Ridley, John Boyega, Oscar Isaac, Domhnall Gleeson",2015,136,8.1,661608,936.63,81
115,Harry Potter and the Deathly Hallows: Part 2,"Adventure,Drama,Fantasy","Harry, Ron and Hermione search for Voldemort's remaining Horcruxes in their effort to destroy the Dark Lord as the final battle rages on at Hogwarts.",David Yates,"Daniel Radcliffe, Emma Watson, Rupert Grint, Michael Gambon",2011,130,8.1,590595,380.96,87
78,Inglourious Basterds,"Adventure,Drama,War","In Nazi-occupied France during World War II, a plan to assassinate Nazi leaders by a group of Jewish U.S. soldiers coincides with a theatre owner's vengeful plans for the same.",Quentin Tarantino,"Brad Pitt, Diane Kruger, Eli Roth,MÃ©lanie Laurent",2009,153,8.3,959065,120.52,69
695,Life of Pi,"Adventure,Drama,Fantasy","A young man who survives a disaster at sea is hurtled into an epic journey of adventure and discovery. While cast away, he forms an unexpected connection with another survivor: a fearsome Bengal tiger.",Ang Lee,"Suraj Sharma, Irrfan Khan, Adil Hussain, Tabu",2012,127,7.9,471109,124.98,79
610,Hellboy II: The Golden Army,"Action,Adventure,Fantasy","The mythical world starts a rebellion against humanity in order to rule the Earth, so Hellboy and his team must save the world from the rebellious creatures.",Guillermo del Toro,"Ron Perlman, Selma Blair, Doug Jones, John Alexander",2008,120,7,216932,75.75,78


<h3>Cleanup</h3>
<p>This section just cleans up the intermediate extracted data file.</p>

In [24]:
#%% Cleanup files
cd.cleanupFile(EXTRACTED_DATA_PATH)